In [12]:
import pandas as pd
import numpy as np
from pandas.api.types import is_float_dtype,is_int64_dtype
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

energy = pd.read_csv("energy_dataset.csv")

# print(energy.info())
# for el in energy.columns:
#     if(el != "time"):
#         print(el, energy[el].mean(), energy[el].std())

green_energy = energy[["time","generation solar","generation wind onshore","generation hydro pumped storage consumption","generation hydro run-of-river and poundage","generation hydro water reservoir","generation nuclear","generation other renewable"]]
green_energy['time'] = pd.to_datetime(green_energy['time'], utc=True, infer_datetime_format=True) +  pd.Timedelta(hours=1)
green_energy = green_energy.set_index('time')
green_energy.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

green_energy.describe()

c:\Users\Cata\anaconda3\envs\UC\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,generation solar,generation wind onshore,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other renewable
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000
mean,1432.818546,5464.980450,475.582706,972.201902,2605.534123,6263.483430,85.634326
std,1679.961733,3213.586296,792.312704,400.712304,1835.175078,840.272333,14.076948
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,71.000000,2933.000000,0.000000,637.000000,1078.000000,5759.000000,73.000000
50%,616.000000,4849.500000,68.000000,906.000000,2165.000000,6564.000000,88.000000
75%,2579.000000,7399.500000,616.000000,1250.000000,3758.000000,7025.000000,97.000000
max,5792.000000,17436.000000,4523.000000,2000.000000,9728.000000,7117.000000,119.000000


In [11]:
green_energy.to_csv("green.csv")

In [5]:

weather = pd.read_csv("weather_features.csv")

weather = weather[["dt_iso","city_name","temp","temp_min","temp_max","pressure","humidity","wind_speed","rain_1h","clouds_all"]]
weather['time'] = pd.to_datetime(weather['dt_iso'], utc=True, infer_datetime_format=True) +  pd.Timedelta(hours=1)
weather = weather.drop(['dt_iso'], axis=1)
weather = weather.set_index('time')

cols = ["temp","temp_min","temp_max","pressure","humidity","wind_speed","rain_1h","clouds_all"]
for col in cols:
    weather[col] = weather[col].values.astype(np.float64)

print(weather.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 178396 entries, 2015-01-01 00:00:00+00:00 to 2018-12-31 23:00:00+00:00
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   city_name   178396 non-null  object 
 1   temp        178396 non-null  float64
 2   temp_min    178396 non-null  float64
 3   temp_max    178396 non-null  float64
 4   pressure    178396 non-null  float64
 5   humidity    178396 non-null  float64
 6   wind_speed  178396 non-null  float64
 7   rain_1h     178396 non-null  float64
 8   clouds_all  178396 non-null  float64
dtypes: float64(8), object(1)
memory usage: 13.6+ MB
None


In [6]:
mean_weather_by_city = weather.groupby('city_name').mean()
mean_weather_by_city

,temp,temp_min,temp_max,pressure,humidity,wind_speed,rain_1h,clouds_all
city_name,,,,,,,,
Barcelona,289.848248,288.594704,291.021987,1284.010486,73.994221,2.786588,0.117079,23.229648
Bilbao,286.378489,284.916661,288.036687,1017.567439,79.089455,1.957470,0.123493,43.960697
Madrid,288.061071,286.824877,289.155600,1011.838448,59.776932,2.441696,0.055083,22.397028
Seville,293.105431,291.184103,295.962431,1018.504711,64.140732,2.483787,0.045392,14.748770
Valencia,290.780780,290.222277,291.355025,1015.973794,65.145113,2.692815,0.035924,20.820999


In [7]:
# Display the number of rows in each dataframe

print('There are {} observations in df_energy.'.format(energy.shape[0]))

cities = weather['city_name'].unique()
grouped_weather = weather.groupby('city_name')

for city in cities:
    print('There are {} observations in df_weather'
          .format(grouped_weather.get_group('{}'.format(city)).shape[0]), 
          'about city: {}.'.format(city))




There are 35064 observations in df_energy.
There are 35145 observations in df_weather about city: Valencia.
There are 36267 observations in df_weather about city: Madrid.
There are 35951 observations in df_weather about city: Bilbao.
There are 35476 observations in df_weather about city:  Barcelona.
There are 35557 observations in df_weather about city: Seville.


In [8]:
weather = weather.reset_index().drop_duplicates(subset=['time', 'city_name'],
                                                      keep='first').set_index('time')

In [9]:
print('There are {} observations in df_energy.'.format(energy.shape[0]))

cities = weather['city_name'].unique()
grouped_weather = weather.groupby('city_name')

for city in cities:
    print('There are {} observations in df_weather'
          .format(grouped_weather.get_group('{}'.format(city)).shape[0]), 
          'about city: {}.'.format(city))


There are 35064 observations in df_energy.
There are 35064 observations in df_weather about city: Valencia.
There are 35064 observations in df_weather about city: Madrid.
There are 35064 observations in df_weather about city: Bilbao.
There are 35064 observations in df_weather about city:  Barcelona.
There are 35064 observations in df_weather about city: Seville.


In [10]:

final = grouped_weather.get_group("Valencia")[["temp","temp_min","temp_max","pressure","humidity","wind_speed","rain_1h","clouds_all"]]
l = ["temp","pressure","humidity","wind_speed","rain_1h","clouds_all"]
print(final)
for city in cities:
    if(city != "Valencia"):
        for el in l:
            final[el] += grouped_weather.get_group(city)[el]
        final["temp_min"] = 1/2 * (final["temp_min"] + grouped_weather.get_group(city)["temp_min"] - np.abs(final["temp_min"]-grouped_weather.get_group(city)["temp_min"]))
        final["temp_max"] = 1/2 * (final["temp_max"] + grouped_weather.get_group(city)["temp_max"] + np.abs(final["temp_max"]-grouped_weather.get_group(city)["temp_max"]))
for el in l:
    final[el] /= 5 
print(final)

                              temp  temp_min  temp_max  pressure  humidity  \
time                                                                         
2015-01-01 00:00:00+00:00  270.475   270.475   270.475    1001.0      77.0   
2015-01-01 01:00:00+00:00  270.475   270.475   270.475    1001.0      77.0   
2015-01-01 02:00:00+00:00  269.686   269.686   269.686    1002.0      78.0   
2015-01-01 03:00:00+00:00  269.686   269.686   269.686    1002.0      78.0   
2015-01-01 04:00:00+00:00  269.686   269.686   269.686    1002.0      78.0   
...                            ...       ...       ...       ...       ...   
2018-12-31 19:00:00+00:00  285.640   285.150   286.150    1028.0      62.0   
2018-12-31 20:00:00+00:00  283.140   282.150   284.150    1029.0      71.0   
2018-12-31 21:00:00+00:00  281.660   281.150   282.150    1029.0      81.0   
2018-12-31 22:00:00+00:00  280.140   279.150   281.150    1029.0      81.0   
2018-12-31 23:00:00+00:00  279.140   278.150   280.150    1029.0

In [15]:
final.to_csv("weather.csv")